# TEST STATISTICAL SIGNIFICANCE

This notebook calculates statistical significance for 4 core metrics:

- `add_payment_info / session`
- `add_shipping_info / session`
- `begin_checkout / session`
- `new_accounts / session`

###  Method:
- For each test and metric:
  - Calculate conversion rates for A and B groups
  - Run a Z-test for proportions
  - Determine if the difference is statistically significant (`p < 0.05`)
- Works for any number of test cases using loops

###  Output (per row):
Each result is returned as a dictionary with:
- `test_number`, `metrics`, `numerator_event`, `denominator`
- `numerator_count_A`, `denominator_count_A`, `conversion_rate_A`
- `numerator_count_B`, `denominator_count_B`, `conversion_rate_B`
- `z-stat`, `p_value`, `significant`

This output is used later for Tableau visualization.

# 1. INSTALLATION, LIBRARY IMPORTS & SQL CONNECTION

In [20]:
!pip install --upgrade google-cloud-bigquery pandas --quiet

In [21]:
# import libraries and modules

import pandas as pd
import numpy as np

from google.cloud import bigquery
from google.colab import auth
from statsmodels.stats.proportion import proportions_ztest

# Authenticating Google Cloud access in Google Colab

auth.authenticate_user()

# Initializing the BigQuery Client and Executing a SQL Query
# To enable further statistical analysis, we add a column with all session dates

client = bigquery.Client(project='data-analytics-mate')
query = """
WITH
  session_info AS(
  SELECT
    s.date,
    ab.ga_session_id,
    sp.channel,
    sp.continent,
    sp.country,
    sp.device,
    ab.test,
    ab.test_group
  FROM
    `data-analytics-mate.DA.ab_test` ab
  JOIN
    `DA.session` s
  ON
    ab.ga_session_id = s.ga_session_id
  JOIN
    `data-analytics-mate.DA.session_params` sp
  ON
    ab.ga_session_id = sp.ga_session_id),
  session_with_order AS (
  SELECT
    session_info.date,
    session_info.channel,
    session_info.continent,
    session_info.country,
    session_info.device,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT o.ga_session_id) AS session_with_order
  FROM
    `DA.order` o
  JOIN
    session_info
  ON
    o.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.channel,
    session_info.continent,
    session_info.country,
    session_info.device,
    session_info.test,
    session_info.test_group),
  events AS (
  SELECT
    session_info.date,
    session_info.channel,
    session_info.continent,
    session_info.country,
    session_info.device,
    session_info.test,
    session_info.test_group,
    event_name,
    COUNT(ep.ga_session_id) AS event_count
  FROM
    `data-analytics-mate.DA.event_params` ep
  JOIN
    session_info
  ON
    ep.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.channel,
    session_info.continent,
    session_info.country,
    session_info.device,
    session_info.test,
    session_info.test_group,
    event_name ),
  sessions AS(
  SELECT
    session_info.date,
    session_info.channel,
    session_info.continent,
    session_info.country,
    session_info.device,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT session_info.ga_session_id) AS session_cnt
  FROM
    session_info
  GROUP BY
    session_info.date,
    session_info.channel,
    session_info.continent,
    session_info.country,
    session_info.device,
    session_info.test,
    session_info.test_group ),
  accounts AS (
  SELECT
    session_info.date,
    session_info.channel,
    session_info.continent,
    session_info.country,
    session_info.device,
    session_info.test,
    session_info.test_group,
    COUNT(acs.account_id) AS new_account
  FROM
    `data-analytics-mate.DA.account_session` acs
  JOIN
    session_info
  ON
    acs.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.channel,
    session_info.continent,
    session_info.country,
    session_info.device,
    session_info.test,
    session_info.test_group )
SELECT
  session_with_order.date,
  session_with_order.channel,
  session_with_order.continent,
  session_with_order.country,
  session_with_order.device,
  session_with_order.test,
  session_with_order.test_group,
  'session with order' AS event_name,
  session_with_order.session_with_order AS value
FROM
  session_with_order
UNION ALL
SELECT
  events.date,
  events.channel,
  events.continent,
  events.country,
  events.device,
  events.test,
  events.test_group,
  event_name,
  events.event_count
FROM
  events
UNION ALL
SELECT
  sessions.date,
  sessions.channel,
  sessions.continent,
  sessions.country,
  sessions.device,
  sessions.test,
  sessions.test_group,
  'sessions',
  sessions.session_cnt
FROM
  sessions
UNION ALL
SELECT
  accounts.date,
  accounts.channel,
  accounts.continent,
  accounts.country,
  accounts.device,
  accounts.test,
  accounts.test_group,
  'new_account',
  accounts.new_account
FROM
  accounts
"""

In [22]:
# Run the query using BigQuery client and convert the result to a pandas DataFrame

df = client.query(query).to_dataframe()
df.head()

,date,channel,continent,country,device,test,test_group,event_name,value
0,2020-11-01,Paid Search,Europe,Lithuania,desktop,2,2,new_account,1
1,2020-11-03,Direct,Americas,Ecuador,mobile,2,2,new_account,1
2,2020-11-05,Direct,Asia,Lebanon,mobile,2,2,new_account,1
3,2020-11-05,Organic Search,Europe,Belarus,desktop,2,1,new_account,1
4,2020-11-08,Direct,Asia,Lebanon,mobile,2,1,new_account,1


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        800996 non-null  dbdate
 1   channel     800996 non-null  object
 2   continent   800996 non-null  object
 3   country     800996 non-null  object
 4   device      800996 non-null  object
 5   test        800996 non-null  Int64 
 6   test_group  800996 non-null  Int64 
 7   event_name  800996 non-null  object
 8   value       800996 non-null  Int64 
dtypes: Int64(3), dbdate(1), object(5)
memory usage: 57.3+ MB


# 2. DATA CLEANING

In [24]:
df['date']= pd.to_datetime(df['date'], format= '%Y-%m-%d')

In [25]:
df['channel']= df['channel'].str.strip().str.lower()
df['continent']= df['continent'].str.strip().str.lower()
df['country']= df['country'].str.strip().str.lower()
df['device'] = df['device'].str.strip().str.lower()
df['event_name'] = df['event_name'].str.strip().str.lower()
df.isna().sum()
df_cleaned = df.copy()
df_cleaned = df.drop_duplicates(df)
df_cleaned.info()
df_cleaned.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        800996 non-null  datetime64[ns]
 1   channel     800996 non-null  object        
 2   continent   800996 non-null  object        
 3   country     800996 non-null  object        
 4   device      800996 non-null  object        
 5   test        800996 non-null  Int64         
 6   test_group  800996 non-null  Int64         
 7   event_name  800996 non-null  object        
 8   value       800996 non-null  Int64         
dtypes: Int64(3), datetime64[ns](1), object(5)
memory usage: 57.3+ MB


,date,channel,continent,country,device,test,test_group,event_name,value
0,2020-11-01,paid search,europe,lithuania,desktop,2,2,new_account,1
1,2020-11-03,direct,americas,ecuador,mobile,2,2,new_account,1
2,2020-11-05,direct,asia,lebanon,mobile,2,2,new_account,1
3,2020-11-05,organic search,europe,belarus,desktop,2,1,new_account,1
4,2020-11-08,direct,asia,lebanon,mobile,2,1,new_account,1


In [26]:
df_cleaned = df.copy()
df_cleaned = df.drop_duplicates(df)
df_cleaned['channel'] = df_cleaned['channel'].str.title()
df_cleaned['continent'] = df_cleaned['continent'].str.title()
df_cleaned['country'] = df_cleaned["country"].str.title()
df_cleaned['device'] = df_cleaned["device"].str.title()
df_cleaned_event_name = df_cleaned.groupby('event_name')['value'].count().sort_values(ascending = False)
print(df_cleaned_event_name)

df_cleaned.head()

event_name
sessions               107210
session_start          106242
page_view              101907
user_engagement         94520
first_visit             81621
scroll                  73643
view_promotion          61695
view_item               44869
session with order      25892
new_account             22389
view_search_results     14310
add_shipping_info       11961
begin_checkout          11959
select_item             11816
add_to_cart             11762
select_promotion         9044
add_payment_info         8048
click                    1997
view_item_list            111
Name: value, dtype: Int64


,date,channel,continent,country,device,test,test_group,event_name,value
0,2020-11-01,Paid Search,Europe,Lithuania,Desktop,2,2,new_account,1
1,2020-11-03,Direct,Americas,Ecuador,Mobile,2,2,new_account,1
2,2020-11-05,Direct,Asia,Lebanon,Mobile,2,2,new_account,1
3,2020-11-05,Organic Search,Europe,Belarus,Desktop,2,1,new_account,1
4,2020-11-08,Direct,Asia,Lebanon,Mobile,2,1,new_account,1


In [27]:
session = len(df_cleaned['event_name'])
print(session)

800996


# 3. CALCULATING STATISTICAL SIGNIFICANCE

In [28]:
def test_analysis(df, test_number, metric, denominator, filters = None):
  """
    Perform z-test for conversion rates between two test groups for a given metric.

    Args:
        df (DataFrame): Original data.
        test_number (int): Number of the AB test.
        metric (str): Event name to analyze (numerator).
        denominator (str): Event name for denominator (e.g., sessions)
        filters (str, optional): Additional filtering condition expressed as a pandas query string
                                 (e.g., "country == 'USA' and device == 'mobile'").
                                 Default is None (no additional filtering).

    Returns:
        dict: Dictionary with test results including conversion rates, z-stat, p-value, and significance.
    """

  #  Filter data for the selected test

  df_test = df[df['test'] == test_number]

  # Apply additional filtering based on key-value pairs if provided

  if filters:
    for key, value in filters.items():
      df_test = df_test[df_test[key] == value]

  # Calculate counts and rates in each groups
  numerator_A = df_test[(df_test['event_name'] == metric) & (df_test['test_group'] == 1)]['value'].sum()
  numerator_B = df_test[(df_test['event_name'] == metric) & (df_test['test_group'] == 2)]['value'].sum()

  denominator_A = df_test[(df_test['event_name'] == denominator) & (df_test['test_group'] == 1)]['value'].sum()
  denominator_B = df_test[(df_test['event_name'] == denominator) & (df_test['test_group'] == 2)]['value'].sum()

  conversion_rate_A = round(numerator_A / denominator_A, 4) if denominator_A != 0 else 0
  conversion_rate_B = round(numerator_B / denominator_B, 4) if denominator_B != 0 else 0

  # Determine z_stat and p_value if denominator values is not 0
  if denominator_A == 0 or denominator_B == 0:
    z_stat, p_value = float('nan'), float('nan')
    significant = None

  else: z_stat, p_value = proportions_ztest([numerator_A, numerator_B], [denominator_A, denominator_B])

  # interpret the test results
  if pd.isna(p_value):
    significant = None
  else:
    significant = p_value < 0.05

  return {"test_number": test_number,
          "metrics": metric,
          "numerator_event": metric,
          "denominator": denominator,
          "numerator_count_A": numerator_A,
          "denominator_count_A": denominator_A,
          "conversion_rate_A": conversion_rate_A,
          "numerator_count_B": numerator_B,
          "denominator_count_B": denominator_B,
          "conversion_rate_B": conversion_rate_B,
          "z-stat": z_stat,
          "p_value": p_value,
          "significant": significant
          }

results = []

# Calculate AB test results for each main metric and each test number

metrics = ["add_payment_info", "add_shipping_info",  "begin_checkout", 'new_account']
denominator = "sessions"
test_number = [1, 2, 3, 4]

for metric in metrics:
  for number in test_number:
    result = test_analysis(df_cleaned, number, metric, denominator)
    results.append(result)

df_results = pd.DataFrame(results)
df_results.head(24)



,test_number,metrics,numerator_event,denominator,numerator_count_A,denominator_count_A,conversion_rate_A,numerator_count_B,denominator_count_B,conversion_rate_B,z-stat,p_value,significant
0,1,add_payment_info,add_payment_info,sessions,1988,45362,0.0438,2229,45193,0.0493,-3.924884,0.000087,True
1,2,add_payment_info,add_payment_info,sessions,2344,50637,0.0463,2409,50244,0.0479,-1.240994,0.214608,False
2,3,add_payment_info,add_payment_info,sessions,3623,70047,0.0517,3697,70439,0.0525,-0.643172,0.520112,False
3,4,add_payment_info,add_payment_info,sessions,3731,105079,0.0355,3601,105141,0.0342,1.571106,0.116158,False
4,1,add_shipping_info,add_shipping_info,sessions,3034,45362,0.0669,3221,45193,0.0713,-2.603571,0.009226,True
5,2,add_shipping_info,add_shipping_info,sessions,3480,50637,0.0687,3510,50244,0.0699,-0.709557,0.477979,False
6,3,add_shipping_info,add_shipping_info,sessions,5298,70047,0.0756,5188,70439,0.0737,1.413727,0.157442,False
7,4,add_shipping_info,add_shipping_info,sessions,5128,105079,0.0488,4956,105141,0.0471,1.785795,0.074132,False
8,1,begin_checkout,begin_checkout,sessions,3784,45362,0.0834,4021,45193,0.0890,-2.978783,0.002894,True
9,2,begin_checkout,begin_checkout,sessions,4262,50637,0.0842,4313,50244,0.0858,-0.952898,0.340642,False


**CONCLUSION**


This notebook provides an automated and robust approach to evaluating the statistical significance of A/B tests for four key funnel metrics: add_payment_info / session, add_shipping_info / session, begin_checkout / session, and new_accounts / session.

Utilizing a Z-test for proportions, the notebook systematically calculates conversion rates for both control (A) and test (B) groups for each test and metric. The analysis results, including z-stat, p_value, and significant determination, are returned in a structured dictionary, making them ready for further visualization in Tableau. This approach enables quick and scalable determination of whether observed differences between variants are statistically significant (at p < 0.05), thereby supporting data-driven decision-making.

# 4. Export CSV file for further analysis in Tableau

In [29]:
csv_file_name = 'ab_test_e_commerce.csv'
df_results.to_csv(csv_file_name, index= False , encoding='utf-8-sig', sep=';')
from google.colab import files
files.download(csv_file_name)

print(f"File '{csv_file_name}' saved and ready for download")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File 'ab_test_e_commerce.csv' saved and ready for download


https://public.tableau.com/views/ABTestAnalysis_17504420018160/ABtest?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link